This notebook is used to use KIM to perform inverse modeling for the cloud chamber model.

In [1]:
# Libraries
from pathlib import Path
import pandas as pd
import numpy as np

from kim.map import KIM
from kim.data import Data
from kim.mapping_model import MLP

import jax

%load_ext autoreload
%autoreload 2


In [2]:
jax.devices()

[CpuDevice(id=0)]

# Read the data

In [3]:
# File and folder paths
f_para = Path("./data/Output_512.csv")
f_state = Path("./data/Input_512.csv")


In [4]:
df_para, df_state = pd.read_csv(f_para),pd.read_csv(f_state)

In [5]:
y_keys, x_keys = df_para.keys(), df_state.keys()
y, x = df_para.values, df_state.values

In [6]:
x.shape, y.shape

((513, 1458), (513, 2))

# Configurations

## Preliminary analysis configuration

In [7]:
seed_shuffle = 1234
f_data_save = Path("./results/data-corr")


In [8]:
# Data configuration
data_params = {
    "xscaler_type": "standard",
    "yscaler_type": "standard",
}

# Sensitivity analysis configuration
sensitivity_params = {
    "method": "pc", "metric": "corr",
    "sst": True, "ntest": 100, "alpha": 0.05, "k": 3,
    "n_jobs": 100, "seed_shuffle": seed_shuffle,
    "verbose": 1
}


## Ensemble learning configuration

In [15]:
Ns_train = 400
hidden_activation = 'sigmoid'
final_activation = 'leaky_relu'
seed_ens = 1024
seed_predict = 3636
seed_dl = 10
seed_model = 100
training_verbose = 1
n_models = 100

f_kim_save1 = Path("./results/map_many2many-corr")
f_kim_save2 = Path("./results/map_many2one-corr")
f_kim_save3 = Path("./results/map_many2one_cond-corr")


In [10]:
# Mapping parameters for each test below
map_configs = {
    "model_type": MLP,
    'n_model': n_models,
    'ensemble_type': 'ens_random',
    'model_hp_choices': {
        "depth": [1,3,5,6],
        "width_size": [3,6,10]
    },
    'model_hp_fixed': {
        "hidden_activation": hidden_activation,
        "final_activation": final_activation,
        "model_seed": seed_model
    },
    'optax_hp_choices': {
        'learning_rate': [0.01, 0.005, 0.003],
    },
    'optax_hp_fixed': {
        'nsteps': 300,
        'optimizer_type': 'adam',
    },
    'dl_hp_choices': {
    },
    'dl_hp_fixed': {
        'dl_seed': seed_dl,
        'num_train_sample': Ns_train,
        'batch_size': 64
    },
    'ens_seed': seed_ens,
    'training_parallel': True,
    'parallel_config': {
        'n_jobs': n_models, 
        'backend': 'loky',
        'verbose': 1
    },
    'device': None,
}

# Exploratory data analysis

In [11]:
data = Data(x, y, **data_params)
data.calculate_sensitivity(**sensitivity_params)


Using correlation metrics ...
Performing pairwise analysis to remove insensitive inputs ...


/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Performing conditional independence testing to remove redundant inputs ...


In [12]:
# Save the sensitivity analysis to disk
data.save(f_data_save)

In [ ]:
# data.sensitivity_mask
# data.cond_sensitivity_mask

# Train the inverse mapping

In [13]:
d = jax.numpy.array([0,3,5,7])
d.devices()

{CpuDevice(id=0)}

In [14]:
# Initialize three diffferent KIMs
kim1 = KIM(data, map_configs, map_option='many2many')
kim2 = KIM(data, map_configs, mask_option="sensitivity", map_option='many2one')
kim3 = KIM(data, map_configs, mask_option="cond_sensitivity", map_option='many2one')

# Train the mappings
kim1.train()
kim2.train()
kim3.train()



 Performing ensemble training in parallel with 100 model configurations...



[Parallel(n_jobs=100)]: Using backend LokyBackend with 100 concurrent workers.
/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
100%|██████████| 300/300 [00:05<00:00, 52.64it/s]
[Parallel(n_jobs=100)]: Done 100 out of 100 | elapsed:  2.9min finished
[Parallel(n_jobs=100)]: Using backend LokyBackend with 100 concurrent workers.


Training completes.

 Performing ensemble training in parallel with 100 model configurations...



100%|██████████| 300/300 [00:06<00:00, 48.48it/s]

100%|██████████| 300/300 [00:06<00:00, 46.80it/s]
[Parallel(n_jobs=100)]: Done 100 out of 100 | elapsed:    8.1s finished
[Parallel(n_jobs=100)]: Using backend LokyBackend with 100 concurrent workers.


Training completes.

 Performing ensemble training in parallel with 100 model configurations...



...

[Parallel(n_jobs=100)]: Using backend LokyBackend with 100 concurrent workers.
100%|██████████| 300/300 [00:06<00:00, 46.75it/s]
[Parallel(n_jobs=100)]: Done 100 out of 100 | elapsed:    7.2s finished
[Parallel(n_jobs=100)]: Using backend LokyBackend with 100 concurrent workers.


Training completes.

 Performing ensemble training in parallel with 100 model configurations...



100%|██████████| 300/300 [00:05<00:00, 57.81it/s]


Training completes.


100%|██████████| 300/300 [00:04<00:00, 60.33it/s]
[Parallel(n_jobs=100)]: Done 100 out of 100 | elapsed:    5.9s finished


In [16]:
# Save 
kim1.save(f_kim_save1)
kim2.save(f_kim_save2)
kim3.save(f_kim_save3)
